## RULE PARSING

In [127]:
import lingua

LANG = "pl"

# LANG_OBJ = getattr(lingua.isocode.IsoCode639_1, LANG.upper())
LANG_OBJ = lingua.Language.POLISH

In [2]:
with open(r"C:\dev\razumlivost\src\flavorizers\{}.ts".format(LANG), "r", encoding="utf8") as f:
    RULES = f.read()

In [3]:
import sys
sys.path.append("C:\\Users\\79165\\anaconda3\\envs\\slovnik\\lib\\site-packages\\")

sys.path.append("C://dev/ISV_data_gathering/")

from translation_aux import prepare_slovnik
from isv_translate import get_slovnik

from isv_nlp_utils import constants
from isv_translate import translate_sentence, postprocess_translation_details, prepare_parsing

In [4]:
for i, line in enumerate(RULES.split("\n")):
    if "multireplacer" in line and "import" not in line:
        break

RULES = "\n".join(RULES.split("\n")[i:])

In [5]:
RULES = "\n".join(l for l in RULES.split("\n") if not l.strip().startswith("//"))

In [6]:
from pyparsing import Group, Opt, Literal, Word, sgl_quoted_string, Suppress
import pyparsing

L = pyparsing.Literal
S = Suppress

chars = pyparsing.unicode.alphas

In [37]:
SS = "        c: 'a',"

DICT_ENTRY.parseString(SS)

ParseResults(['c', 'a'], {})

In [83]:
DICT_ENTRY = (
    (Word(chars) | "'’'")+ 
    L(":").suppress() + 
    L("'").suppress() + 
    Opt(Word(chars), default="") + 
    L("'").suppress() + 
    L(",").suppress()
)

SS = "        c: '',"

DICT_ENTRY.parseString(SS)

ParseResults(['c', ''], {})

In [23]:
SS[14:]

"',"

In [84]:
DICT_ENTRY = (
    (Word(chars) | "'’'")+ 
    L(":").suppress() + 
    L("'").suppress() + 
    Opt(Word(chars), default="") + 
    L("'").suppress() + 
    L(",").suppress()
)

MAP_DESC = (
    S("{") + 
    pyparsing.OneOrMore(
        pyparsing.Group(DICT_ENTRY)
    )("map") + 
    S("}")
)

RULE_CASE =  L("(r) => r.lowerCase()") | L("(r) => r.restoreCase()")
RULE_REGEX = S("(r)") + S("=>") + "r.regexp" + S("(/") + ... + S("/") + S(",") + S('[') + Group(pyparsing.OneOrMore(sgl_quoted_string + Opt(",").suppress())) + S(']') + S(")")
RULE_MAP =   S("(r)") + S("=>") + "r.map" + S("(") + MAP_DESC + S(")")
CONSTRAINT = S("(p)") + S("=>") + S("p.") + Group(Word(chars) + S("(") + ... + S(")")) + S(",")


rule_content = sgl_quoted_string + S(",") + (RULE_REGEX | RULE_MAP | RULE_CASE) + Opt(",").suppress() + Opt(CONSTRAINT) + Opt(",").suppress()



In [85]:
rule_expr     = pyparsing.nestedExpr( '(', ')', content=rule_content)


element = (
    Group(Suppress(".rule") + S("(") + rule_content + S(")"))("rule") | 
    Group(Literal(".named") + S("(") + sgl_quoted_string  + S(")"))("named") | 
    Group(Literal(".section") + S("(") + sgl_quoted_string  + S(")"))("section")
)

In [86]:
S1 = RULES[2801:]
print(S1[:200])

element.parseString(S1)

.rule('Etymological → Polish', (r) =>
      r.map({
        ę: 'ę',
        å: 'o',
        ğ: 'h',
        ė: 'e',
        ȯ: 'e',
        đ: 'dz',
        ĺ: 'l',
        ď: 'dź',
        ḓ: 'd',
  


ParseResults([ParseResults(["'Etymological → Polish'", 'r.map', ParseResults(['ę', 'ę'], {}), ParseResults(['å', 'o'], {}), ParseResults(['ğ', 'h'], {}), ParseResults(['ė', 'e'], {}), ParseResults(['ȯ', 'e'], {}), ParseResults(['đ', 'dz'], {}), ParseResults(['ĺ', 'l'], {}), ParseResults(['ď', 'dź'], {}), ParseResults(['ḓ', 'd'], {}), ParseResults(['ś', 'ś'], {}), ParseResults(['ù', 'u'], {}), ParseResults(['ť', 't'], {}), ParseResults(['ź', 'ź'], {}), ParseResults(["'’'", ''], {})], {'map': [['ę', 'ę'], ['å', 'o'], ['ğ', 'h'], ['ė', 'e'], ['ȯ', 'e'], ['đ', 'dz'], ['ĺ', 'l'], ['ď', 'dź'], ['ḓ', 'd'], ['ś', 'ś'], ['ù', 'u'], ['ť', 't'], ['ź', 'ź'], ["'’'", '']]})], {'rule': {'map': [['ę', 'ę'], ['å', 'o'], ['ğ', 'h'], ['ė', 'e'], ['ȯ', 'e'], ['đ', 'dz'], ['ĺ', 'l'], ['ď', 'dź'], ['ḓ', 'd'], ['ś', 'ś'], ['ù', 'u'], ['ť', 't'], ['ź', 'ź'], ["'’'", '']]}})

In [87]:
parser = S("multireplacer") + pyparsing.ZeroOrMore(element) + L(".build();")

In [88]:
res = parser.parseString(RULES)



# TOKENIZATION

In [89]:
import pandas as pd


slovnik = get_slovnik()
slovnik = slovnik['words']


Found 'slovnik.pkl' file, using it


In [90]:
from razdel import tokenize
import re

In [91]:
import unicodedata

def convert2MSPlus(thestring):
    # "e^" -> "ê"
    # 'z\u030C\u030C\u030C' -> 'ž\u030C\u030C'
    thestring = unicodedata.normalize(
        'NFKC',
        thestring
    )
    # TODO: do the same for upper case
    thestring = (
        thestring
        .replace("ň", "ń").replace("ĺ", "ľ")
        .replace("d\u0301", "ď").replace("t\u0301", "ť")
        .replace("l\u0301", "ľ").replace("n\u0301", "ń")
        .replace("ò", "ȯ").replace("è", "ė")
        .replace("ǉ", "lj").replace("ĳ", "ij").replace("ǌ", "nj").replace("t́", "ť").replace("d́", "ď")
    )
    return thestring



In [92]:
morph = constants.create_analyzers_for_every_alphabet(r"C:\dev\ISV_data_gathering\\")['etm']
morph.char_substitutes['ė'.encode("utf8")] = ('e'.encode("utf8"), 'e')

In [93]:
Src = "Zdråvěj i zdråvějte vsi, prĳateǉi! Věš, možno i v zlom světu, no jestli někto postųpaje toksično i manipuluje drugymi ǉud́ami, potom jest nažalost́ nebezpečny kako člověk. Anton imaje svojų pravdų, uběđaje ǉudĳ s slabějšim harakterom i bez svojego mněńja, a jestli jesi protiv ǌego, potom u ǌego imaješ problem. Prinajmėǌe tako to viđų ja."

Src = convert2MSPlus(Src)
tokens = list(tokenize(Src))


In [94]:
for token in tokens:
    parses = morph.parse(token.text.lower())
    n_forms = set(p.normal_form for p in parses)
    pos_cands = set(p.tag.POS for p in parses)
    N = len(parses)
    
    if N > 1:
        #print(token.text, N, n_forms, pos_cands)
        pass
    for form in n_forms:
        genesis = slovnik[slovnik.isv == form].genesis.dropna().values
        if len(genesis):
            print(token, form, genesis)

Substring(101, 111, 'manipuluje') manipulovati ['I']
Substring(226, 236, 'harakterom') harakter ['I']
Substring(308, 315, 'problem') problem ['I']


In [95]:
from string import punctuation


In [96]:
# from razdel.record import Record
from natasha.record import Record

class AnnotatedToken(Record):
    __attributes__ = ['text', 'capitalization', 'space_after', 'features', 'POS', 'lemma', 'isv_id', 'genesis']


In [97]:
tokens_data = []

for i, t in enumerate(tokens):
    if i + 1 < len(tokens):
        space_after = Src[t.stop: tokens[i+1].start]
    else:
        space_after = ""
    
    parses = morph.parse(t.text.replace("đ", "dʒ"))
    if parses:
        # TODO: HANDLE NON-DETERMINISM HERE
        # BECAUSE I CAN AND WHY NOT
        isv_lemma = parses[0].normal_form
        tag = parses[0].tag
        entry = slovnik[slovnik.isv == isv_lemma]
        pos = str(tag.POS)
        if len(entry):
            isv_id = entry.index[0]
            isv_genesis = entry.genesis.values[0]
        else:
            isv_id = -1
            isv_genesis = ""
    else:
        tag = None
        pos = "PUNCT" if t.text in punctuation else "UNKNOWN"
        isv_lemma = None
        isv_id = -1
        isv_genesis = ""

    ann_token = AnnotatedToken(
        [t.text],
        t.text.istitle(), space_after,
        tag, pos, isv_lemma,
        isv_id, isv_genesis
    )
    tokens_data.append(ann_token)
# is_upper space_after features lemma genesis partOfSpeech



# FLAVORIZATION (MORPHOLOGICAL): RULES

In [98]:

RU_FLAVOR = {
    "VERB":
    {
        "infn": (-1, 'ь'),
        "3per+sing": (None, 't'),
    },
    "NOUN":
    {
        "loct+sing+masc": (-1, "ě"),
    },
    "ADJF":
    {
        "nomn+sing+femn": (None, "ja"),
        "nomn+sing+neut": (None, "ě"),
        "nomn+sing+masc": (None, "j"),

        "accs+sing+femn": (None, "ju"),
        "accs+sing+neut": (None, "ě"),
        "accs+sing+masc+anim": (None, ""),
        "accs+sing+masc+inan": (None, "j"),

        # "accs+plur": (None, "iě"),
        "nomn+plur": (-1, "ые"),
        "accs+plur+neut": (-1, "ые"),
        # "accs+plur+femn+anim": (-1, "ых"),
        # "accs+plur+femn+inan": (-1, "ые"),
        # "accs+plur+masc+anim": (-1, "ых"),
        # "accs+plur+masc+inan": (-1, "ые"),
    },
    "SPECIAL_CASES": 
    {
        "jesm": "являюсь",
        "jesmo": "являемся",
        "jesi": "являешься",
        "jest": "является",
        "jestli": "если"
    }
}

In [99]:

PL_FLAVOR = {
    "VERB":
    {
        "1per+sing": (-1, 'ę'),
        "3per+plur": (-2, 'ą'),
    },
    "NOUN":
    {
        "loct+sing+masc": (-1, "ě"),
        "accs+sing+femn": (-1, "ę"),
    },
    "ADVB": {"ADVB": (-1, 'e')},
    "ADJF":
    {
        "nomn+plur+femn": (-1, "ie"),
        "accs+plur+femn": (-1, "ie"),
        "nomn+plur+neut": (-1, "ie"),
        "accs+plur+neut": (-1, "ie"),

        "accs+sing+femn": (-1, "ą"),
        "ablt+sing+femn": (-1, "ą"),

        "loct+sing+femn": (-2, "ej"),
        "datv+sing+femn": (-2, "ej"),
        "gent+sing+femn": (-2, "ej"),

        "accs+plur+masc": (-2, "ych"),
        "loct+plur": (-2, "ych"),
        "datv+plur": (-2, "ych"),
    },
    "SPECIAL_CASES": {}
}

In [100]:
ju = True

In [101]:
FLAVOR = PL_FLAVOR

# FLAVORIZATION (MORPHOLOGICAL): IMPLEMENTATION

In [102]:


def flavorise(word, golden_pos_tag, isv_morph, flavor, ju):
    if word in flavor["SPECIAL_CASES"]:
        return flavor["SPECIAL_CASES"][word]
    word = word.replace("đ", "dʒ")
    return __flavorise(word, golden_pos_tag, isv_morph, flavor, ju).replace("dʒ", "đ")

def __flavorise(word, golden_pos_tag, isv_morph, flavor, ju):
    if golden_pos_tag == "PNCT":
        return word
    if golden_pos_tag == "ADVB":
        variants = [
            v for v in isv_morph.parse(word)
            if v.tag.POS == "ADJF"
            and v.tag.number == "sing" and v.tag.gender == "neut" and v.tag.case == "nomn"
        ]
        if not variants:
            return word
    else:
        variants = [v for v in isv_morph.parse(word) if golden_pos_tag in v.tag]

    if not variants:
        return word

    if ju:
        if golden_pos_tag == "VERB" and all(v.tag.person == "1per" for v in variants):
            tags = variants[0].tag.grammemes  # no better way to choose
            new_tags = set(tags) - {'V-m'} | {'V-ju'}
            word = isv_morph.parse(word)[0].inflect(new_tags).word

    if golden_pos_tag == "ADJF":
        variants = [variants[0]]  # no better way to choose

    flavor_rules = flavor.get(golden_pos_tag, {})
    if golden_pos_tag == "ADVB":
        flavor_rules = {"ADJF": flavor_rules.get('ADVB', (None, ''))}

    for condition_plus, transform in flavor_rules.items():
        conditions_arr = condition_plus.split("+")
        is_match = all(
            all(cond in v.tag for cond in conditions_arr)
            for v in variants
        )
        if is_match:
            if isinstance(transform, tuple):
                suffix, addition = transform
                return word[:suffix] + addition
            if isinstance(transform, dict):
                for base, replacement in transform.items():
                    if word[-len(base):] == base:
                        return word[:-len(base)] + replacement

    return word




In [103]:
for i, token in enumerate(tokens_data):
    original = token.text[0]
    flavorised = flavorise(token.text[0], token.POS, morph, FLAVOR, ju=ju)
    if original != flavorised:
        print(original, flavorised)
        token.text[0] = flavorised
        # print(token)


možno možne
pravdų pravdę
tako take
viđų viđę


# FLAVORIZATION (BASE): IMPLEMENTATION

In [104]:
# Src = "Zdråvej i zdråvejte vsi, prĳateǉi! Věš, možno i v zlom světu, no jestli někto postųpaje toksično i manipuluje drugymi ǉud́ami, potom jest nažalost́ nebezpečny kako člověk. Anton imaje svojų pravdų, uběđaje ǉudĳ s slabějšim harakterom i bez svojego mněńja, a jestli jesi protiv ǌego, potom u ǌego imaješ problem. Prinajmėǌe tako to viđų ja."

# Src = Src.lower()


for e in res:
    #print(e)
    name, typ = e[:2]
    if "lowerCase" in typ:
        for i in range(len(tokens_data)):
            for j, w in enumerate(tokens_data[i].text):
                tokens_data[i].text[j] = w.lower()
    if "restoreCase" in typ:
        for i in range(len(tokens_data)):
            if tokens_data[i].capitalization:
                for j, w in enumerate(tokens_data[i].text):
                    tokens_data[i].text[j] = w.title()
        
    if typ == "r.map":
        for (a, b) in e[2:]:
            for i in range(len(tokens_data)):
                for j, w in enumerate(tokens_data[i].text):
                    tokens_data[i].text[j] = w.replace(a, b)
    if typ == "r.regexp":
        if len(e) == 4:
            name, typ, pattern, subst = e
            constraint = None
        else:
            name, typ, pattern, subst, constraint = e
            constraint = constraint.asList()
        subst = subst.asList()
        subst = [s.replace("$", "\\").strip("'") for s in subst]
        pattern = pattern.replace("\x08", "$")
        try:
            re.compile(pattern)
        except re.error as e:
            print(e, "in", name)
            continue
        for i in range(len(tokens_data)):
            prev_words = list(tokens_data[i].text)
            candidates = []
            for j, w in enumerate(tokens_data[i].text):
                for one_subst in subst:
                    cand = re.sub(pattern, one_subst, w)
                    candidates.append(cand)
            tokens_data[i].text = list(set(candidates))
            if len(tokens_data[i].text) > len(prev_words):
                print(name)
                print(prev_words, " => ", tokens_data[i].text)
        # print(name, len(candidates), len(set(candidates)))
        # words = choice(candidates)


final = "".join([
    (t.text[0] if len(t.text) == 1 else f"[{'|'.join(t.text)}]") + t.space_after
    for t in tokens_data
])
final

look-behind requires fixed-width pattern in 'Alteration (vòz-)'
look-behind requires fixed-width pattern in 'Alteration (iz-)'
look-behind requires fixed-width pattern in 'Alteration (bez-)'
'-sť'
['nažalosť']  =>  ['nažalośť', 'nažalosť']
'Alterate ending (-ny)'
['nebezpečny']  =>  ['nebezpečovy', 'nebezpečny']
'Zero ending'
['mněńja']  =>  ['mněńja', 'mněńj']
'Zero ending'
['a']  =>  ['', 'a']
'Zero ending'
['ja']  =>  ['j', 'ja']
'-ť → -ć'
['nažalośť', 'nažalosť']  =>  ['nažalosć', 'nažalośť', 'nažalość', 'nažalosť']
'-ć-'
['nažalosć', 'nažalośť', 'nažalość', 'nažalosť']  =>  ['nažalosc', 'nažalosť', 'nažalosć', 'nažalość', 'nažalośc', 'nažalośť']
'Soft R-RZ'
['prijatelji']  =>  ['przijatelji', 'prijatelji']
'Soft R-RZ'
['prinajmėnje']  =>  ['przinajmėnje', 'prinajmėnje']
'T-C'
['zdråviate']  =>  ['zdråviace', 'zdråviate']
'T-C'
['przijatelji', 'prijatelji']  =>  ['przijatelji', 'prijacelji', 'prijatelji', 'przijacelji']
'T-C'
['světu']  =>  ['světu', 'svěcu']
'T-C'
['jestli']  =>  

'[Zdrawyoj|Zdrowej|Zdrawioj|Zdrowaj|Zdrawiej|Zdrawej|Zdrowiej|Zdrawaj|Zdrowyoj|Zdrowyej|Zdrawyej|Zdrowioj] [i|y] [zdrowyace|zdrowiace|zdrowyate|zdrawyace|zdrawyate|zdrawiace|zdrowiate|zdrawiate] [wsi|wsy], [pryjaceljy|prijacełji|pryjacełjy|przyjatełjy|przyjateljy|pryjateljy|przyjacełji|przyjaceljy|przyjatełji|przyjacełjy|prijatelji|przyjatelji|prijatełji|prijacelji|przyjacelji|pryjatełjy]! [Wyosz|Wasz|Wiosz|Wesz|Wyesz|Wiesz], możne [i|y] w [zlom|złom] [swyocu|swiotu|swetu|swatu|swyecu|swiecu|swyotu|swietu|swecu|swyetu|swacu|swiocu], no [jestły|jescli|jestłi|jescły|jescłi|jestly|jescly|jestli] [nyokto|nakto|nakco|nyekto|nyekco|niekco|nekto|nekco|niokto|niokco|niekto|nyokco] [postępaje|postąpaje|poscępaje|poscąpaje] [toksyczno|coksyczno|toksiczno|coksiczno] [i|y] [manipuluje|manypułuje|manypuluje|manipułuje] [drugymy|druchymy|drugymi|druchymi] [łjudźamy|łjudźami|ljudźami|ljudźamy], [potom|pocom] [jesc|jest] [nażalosc|nażałośt|nażalost|nażałost|nażałosc|nażałośc|nażalosć|nażałosć|nażałość

## VARIANT SELECTION

In [106]:
import lingua
lingua.isocode.IsoCode639_3

LANGS = [
    'bel', 'ukr', 'rue', 'rus', 'bos', 'hrv', 'srp', 'cnr', 'slv', 'bul', 'mkd', 'chu', 'wen', 'dsb', 'hsb', 'pol', 'szl', 'csb', 'pox', 'ces', 'czk', 'slk',
]

LANGS = [
    getattr(lingua.isocode.IsoCode639_3, l.upper())
    for l in LANGS
    if l.upper() in dir(lingua.isocode.IsoCode639_3)
]
print(LANGS)

detector = lingua.LanguageDetectorBuilder.from_iso_codes_639_3(*LANGS).build()

[IsoCode639_3.BEL, IsoCode639_3.UKR, IsoCode639_3.RUS, IsoCode639_3.BOS, IsoCode639_3.HRV, IsoCode639_3.SRP, IsoCode639_3.SLV, IsoCode639_3.BUL, IsoCode639_3.MKD, IsoCode639_3.POL, IsoCode639_3.CES, IsoCode639_3.SLK]


In [107]:

def get_conf(confs, lang):
    the_sum = sum(dict(confs).values())
    return {k: v / the_sum for (k, v) in confs}[lang]


In [132]:
final = []
for token in tokens_data:
    w = token.text
    if len(w) > 1:
        scores = {}
        for cand in w:
            if cand:
                confs = detector.compute_language_confidence_values(cand)
                scores[cand] = get_conf(confs, LANG_OBJ)
        print(scores)
        final.append(max(scores, key=lambda x: scores[x]) + token.space_after)
    else:
        final.append(w[0] + token.space_after)

final = "".join(final)
final

{'Zdrawyoj': 0.17823896633345204, 'Zdrowej': 0.2659996736501023, 'Zdrawioj': 0.20585427563265263, 'Zdrowaj': 0.2175662366706651, 'Zdrawiej': 0.23185072942629972, 'Zdrawej': 0.2165312462891608, 'Zdrowiej': 0.24656809719203443, 'Zdrawaj': 0.1951422431865108, 'Zdrowyoj': 0.20134665433305685, 'Zdrowyej': 0.22595992797417583, 'Zdrawyej': 0.1809894104401752, 'Zdrowioj': 0.21701018509113865}
{'i': 0.1694472530987956, 'y': 0.2587136917583172}
{'zdrowyace': 0.19677076951488423, 'zdrowiace': 0.22869669853447247, 'zdrowyate': 0.22238892502274316, 'zdrawyace': 0.17194595000660837, 'zdrawyate': 0.17956705343261312, 'zdrawiace': 0.21040858165520496, 'zdrowiate': 0.25279874214440506, 'zdrawiate': 0.2210346250086031}
{'wsi': 0.21580669380864984, 'wsy': 0.20848875855232818}
{'pryjaceljy': 0.1621665126350064, 'prijacełji': 1.0, 'pryjacełjy': 1.0, 'przyjatełjy': 1.0, 'przyjateljy': 0.16063325604601025, 'pryjateljy': 0.14457046092881268, 'przyjacełji': 1.0, 'przyjaceljy': 0.17670621812643067, 'przyjatełji

'Zdrowej y zdrowiate wsi, prijacełji! Wiesz, możne y w złom swiecu, no jestły niekto postępaje toksyczno y manypułuje druchymy łjudźamy, pocom jest nażalosc nebezpeczowy kako człowyok. Ancon ymaje swoję prawdę, ubiedzaje łjudyj s słabyaszym charakterom y bez swojego mniońja, a jestły jesy protyw njecho, pocom u njecho ymajesz probłem. Przynajmienje cake co widzę ja.'